## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [60]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
# импортируем библиотеку pandas и прочитаем файл с данными
data.info()
# рассмотрим общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [62]:
for row in data:
    print(data[row].value_counts())
#методом value_counts считаем количество дублей ошибок

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

### Шаг 2. Предобработка данных

### Обработка пропусков

In [63]:
data.isnull().sum()# методом isnull.sum узнаем сколько всего недостает     

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [64]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'] /= 365
# избавляемся от отрицательных значений abs модуль числа
# переводим дни стажа в года

In [65]:
data['gender'] = data['gender'].replace('XNA', 'F')
# replace заменит значение XNA на Fб значение одно и не принципиально на какой из гендера его менять

In [66]:
data['dob_years'].min()
# минимальное значение в возрасте

0

In [67]:
data['dob_years'].max()
# максимальное значение в возрасте

75

In [68]:
incommed = data.groupby('income_type')['dob_years'].median()# по каждому типу занятости ищем медиану по возрасту.

In [69]:
incommed

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

In [70]:
# на основании каждого типа занятости заполняем нулевые значения в возрасте на медиану

data.loc[(data['income_type'] == 'безработный') & (data['dob_years'] == 0), 'dob_years'] = 38
data.loc[(data['income_type'] == 'в декрете') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'госслужащий') & (data['dob_years'] == 0), 'dob_years'] = 40
data.loc[(data['income_type'] == 'компаньон') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'пенсионер') & (data['dob_years'] == 0), 'dob_years'] = 60
data.loc[(data['income_type'] == 'предприниматель') & (data['dob_years'] == 0), 'dob_years'] = 42.5
data.loc[(data['income_type'] == 'сотрудник') & (data['dob_years'] == 0), 'dob_years'] = 39
data.loc[(data['income_type'] == 'студент') & (data['dob_years'] == 0), 'dob_years'] = 22

In [71]:
data['dob_years'].isnull().sum()
#проверка: вычисление суммарного количества пропусков, выявленных в таблице

0

In [72]:
data['total_income'] = data['total_income'].fillna(0)
 # заменили пропуски в доходах граждан на нули

In [73]:
#для каждого из типа занятости высчитываем медиану и заменяем значения в доходах на медиану в зависимости от типа дохода
median_s = data['total_income'][data['income_type'] == 'сотрудник'].median()
data.loc[(data['income_type'] == 'сотрудник') & (data['total_income'] == 0),'total_income'] = median_s


In [74]:
median_b = data['total_income'][data['income_type'] == 'безработный'].median()
data.loc[(data['income_type'] == 'безработный') & (data['total_income'] == 0),'total_income'] = median_b

In [75]:
median_d = data['total_income'][data['income_type'] == 'в декрете'].median()
data.loc[(data['income_type'] == 'в декрете') & (data['total_income'] == 0),'total_income'] = median_d

In [76]:
median_g = data['total_income'][data['income_type'] == 'госслужащий'].median()
data.loc[(data['income_type'] == 'госслужащий') & (data['total_income'] == 0),'total_income'] = median_g

In [77]:
median_k = data['total_income'][data['income_type'] == 'компаньон'].median()
data.loc[(data['income_type'] == 'компаньон') & (data['total_income'] == 0),'total_income'] = median_k

In [78]:
median_pr = data['total_income'][data['income_type'] == 'предприниматель'].median()
data.loc[(data['income_type'] == 'предприниматель') & (data['total_income'] == 0),'total_income'] = median_pr
 

In [79]:
median_st = data['total_income'][data['income_type'] == 'студент'].median()
data.loc[(data['income_type'] == 'студент') & (data['total_income'] == 0),'total_income'] = median_st

In [80]:
median_p = data['total_income'][data['income_type'] == 'пенсионер'].median()
data.loc[(data['income_type'] == 'пенсионер') & (data['total_income'] == 0),'total_income'] = median_p

In [81]:
data['children'] = data['children'].replace(20,2)
data['children'] = data['children'].replace(-1,1)
# заменим две ошибки в категории дети

In [82]:
data['days_employed'] = data['days_employed'].fillna(0)
# заменили пропуски в доходах граждан на нули

In [83]:
# группируем данные по возрасту dob_years
# функция принимает как параметр строку с возрастом клиентов 
#в теле объявляется переменная, ей присваивается значение row['dob_years'],
# затем цикл if проходит по списку значений и фильтрует по группам

def days_employed(row):
    age = row['dob_years']
    if age <= 30:
        return '1группа'
    if age > 30 and age <= 40:
        return '2группа'
    if age > 40 and age <= 55:
        return '3группа'
    if age > 55 and age < 80:
        return '4группа'
    else:
        return '5группа'

In [84]:
data['age_group'] = data.apply(days_employed, axis=1)
#в таблице создаем новый столбец показывающий к какой возрастной категории относится человек

In [85]:
data.groupby('age_group')['days_employed'].mean()
#высчитываем средний стаж каждой возрастной группы граждан

age_group
1группа      5.021288
2группа     11.485743
3группа    106.589034
4группа    604.486033
Name: days_employed, dtype: float64

In [86]:
# группируем данные по стажу и в зависимости от принадлежности той или иной возрастной группе и заменяем нули в стаже средним значением 

def group(row):
    group1 = row['age_group']
    group2 = row['days_employed']
    if group2 == 0 and  group1 == '1группа':
        return 5.021288 
   
    if group2 == 0 and  group1 == '2группа':
        return 11.485743
   
    if group2 == 0 and  group1 == '3группа':
        return 106.589034
    if group2 == 0 and  group1 == '4группа':
        return 604.486033
    else: 
        return group2
data['days_employed'] = data.apply(group, axis=1)


In [87]:
data['education'] = data['education'].str.lower() 
#приводим все значения к одному регистру

In [88]:
data['education'].unique() 
# теперь мы видим 5 уникальных значений в столбце образование

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

На этапе предобработки в данных обнаружились и пропуски и нулевые значения в столбцах, а также в столбце education  дубликатов, от которых мы успешно избавились. Их удаление не позволит нам провести точный анализ. Поскольку сведения о доходах,стаже, а также типах занятости важно нам сохранить для анализа, мы не удаляем все пропущенные значения, а заполняем пропущенные значения. Теперь данные таблицы корректны и удобны для работы.

### Замена типа данных

In [89]:
data.dtypes
#рассмотрим какие типы данных в нашей таблице

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_group            object
dtype: object

In [90]:
data['dob_years'] = data['dob_years'].astype('int')

In [91]:
data['total_income'] = data['total_income'].astype('int')

In [92]:
data['days_employed'] = data['days_employed'].astype('int')

подозрительные данные: столбцы 'days_employed', 'dob_years' и 'total_income' распознаны как строки — тип float64. Это может помешать нашим расчётам. с помощью аргумента ('int') метода astype()  переводим эти значения в целое число

### Обработка дубликатов

In [93]:
data.duplicated().sum()
# считаем количество дубликатов в наборе данных

71

In [94]:
data = data.drop_duplicates().reset_index(drop=True)
#удаляем их

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
age_group           21454 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


### Вывод

### Лемматизация

In [96]:
from pymystem3 import Mystem
m = Mystem()
#  импортируется pymystem3

In [97]:
def lemma_pur(purpose):
    lemma = ' '.join(m.lemmatize(purpose))
    return lemma
#Лемматизируем столбец purpose

In [98]:
data['purpose_new'] = data['purpose'].apply(lemma_pur)
#создаем новый столбец purpose_new на основе нашей лемматизации

In [99]:
data['purpose_new'].unique()

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

### Категоризация данных

In [100]:
# группируем данные по доходу  в зависимости от суммы их дохода 
def total_group(row):
    income = row['total_income']
    if income <= 55000:
        return 'доход 15-55'
    if income > 55 and income <= 120000:
        return 'доход 55-120'
    if income > 120000 and income <= 200000:
        return 'доход 120-200'
    else:
        return 'доход больше 200'

In [101]:
data['income_group'] = data.apply(total_group, axis=1)
#создадим столбец содержащий принадлежность данного лица к той или иной группе 

In [102]:
#создадим новый столбец с категориями целей кредита
def purpose_category(list):
     if 'автомобиль' in list:
        return 'автомобиль'
     if 'недвижимость' in list:
        return 'недвижимость'
     if 'жилье' in list:
        return 'жилье'
     if 'образование' in list:
        return 'образование'
     if 'свадьба' in list:
        return 'свадьба'
     if 'строительство' in list:
        return 'строительство'
data['purpose_category'] = data['purpose_new'].apply(purpose_category)

### Вывод

В огромной таблице данных нам понадобиться определённый сегмент,нас интересуют данные по узкой возрастной категории, целям кредита, их доходы. Чтобы выделить их, мы прибегли к категоризации — объединению избранных данных в произвольные группы по заданному критерию.
Для ответа на поставленные задачи таблица data полность готова.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [103]:
data_pivot1 = data.pivot_table(index=['debt'], columns = 'children', values = 'family_status_id', aggfunc = 'count')

In [104]:
 ch0 = data_pivot1[0][1] / data_pivot1[0][0]
ch1 = data_pivot1[1][1] / data_pivot1[1][0]
ch2 = data_pivot1[2][1] / data_pivot1[2][0]
ch3 = data_pivot1[3][1] / data_pivot1[3][0]
ch4 = data_pivot1[4][1] / data_pivot1[4][0]

In [105]:
print("{0:.2f}% нет детей".format(ch0*100))
print("{0:.2f}% 1 ребенок".format(ch1*100))
print("{0:.2f}% 2 детей".format(ch2*100))
print("{0:.2f}% 3 детей".format(ch3*100))
print("{0:.2f}% 4 детей".format(ch4*100))

8.16% нет детей
10.09% 1 ребенок
10.49% 2 детей
8.91% 3 детей
10.81% 4 детей


### Вывод

увеличение колличества детей в семье заемщика в среднем понижает платежную дисциплину и увеличивает риск невозврата кредита

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [106]:
data_pivot2 = data.pivot_table(index=['debt'], columns = 'family_status', values = 'age_group', aggfunc = 'count') 

In [107]:
no_fam = data_pivot2['Не женат / не замужем'][1] / data_pivot2['Не женат / не замужем'][0]
divorce = data_pivot2['в разводе'][1] / data_pivot2['в разводе'][0]
vdow = data_pivot2['вдовец / вдова'][1] / data_pivot2['вдовец / вдова'][0]
gr_br = data_pivot2['гражданский брак'][1] / data_pivot2['гражданский брак'][0]
fam = data_pivot2['женат / замужем'][1] / data_pivot2['женат / замужем'][0]


In [108]:
print("{0:.2f}% Не женат / не замужем".format(no_fam*100))
print("{0:.2f}% в разводе".format(divorce*100))
print("{0:.2f}% вдовец / вдова".format(vdow*100))
print("{0:.2f}% гражданский брак".format(gr_br*100))
print("{0:.2f}% женат / замужем".format(fam*100))

10.80% Не женат / не замужем
7.66% в разводе
7.03% вдовец / вдова
10.31% гражданский брак
8.16% женат / замужем


### Вывод

Респонденты вступавшие в брак, в том числе находящиеся в нем, обладают также повышенной ответственностью, так как имеют опыт исполнения увеличенной финансовой нагрузки при возникновении семьи

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [109]:
data_pivot3 = data.pivot_table(index=['debt'], columns = 'income_group', values = 'age_group', aggfunc = 'count') 

In [110]:
com1 = data_pivot3['доход 15-55'][1] / data_pivot3['доход 15-55'][0]
com2 = data_pivot3['доход 55-120'][1] / data_pivot3['доход 55-120'][0]
com3 = data_pivot3['доход 120-200'][1] / data_pivot3['доход 120-200'][0]
com4 = data_pivot3['доход больше 200'][1] / data_pivot3['доход больше 200'][0]

In [111]:
print("{0:.2f}% доход 15-55".format(com1*100))
print("{0:.2f}% доход 55-120".format(com2*100))
print("{0:.2f}% доход 120-200".format(com3*100))
print("{0:.2f}% доход больше 200".format(com4*100))

6.42% доход 15-55
9.03% доход 55-120
9.53% доход 120-200
7.60% доход больше 200


### Вывод

риск невозврата кредита имеет тенденцию к снижению у респондентов с наиболее низкими доходами и тех чей доход выше среднего

- Как разные цели кредита влияют на его возврат в срок?

In [112]:
data_pivot4 = data.pivot_table(index=['debt'], columns = 'purpose_category', values = 'age_group', aggfunc = 'count') 

In [113]:
cel1 = data_pivot4['недвижимость'][1] / data_pivot4['недвижимость'][0]
cel2 = data_pivot4['образование'][1] / data_pivot4['образование'][0]
cel3 = data_pivot4['автомобиль'][1] / data_pivot4['автомобиль'][0]
cel4 = data_pivot4['свадьба'][1] / data_pivot4['свадьба'][0]
cel5 = data_pivot4['жилье'][1] / data_pivot4['жилье'][0]

In [114]:
print("{0:.2f}% недвижимость".format(cel1*100))
print("{0:.2f}% образование".format(cel2*100))
print("{0:.2f}% автомобиль".format(cel3*100))
print("{0:.2f}% свадьба".format(cel4*100))
print("{0:.2f}% жилье".format(cel5*100))

8.07% недвижимость
10.16% образование
10.33% автомобиль
8.70% свадьба
7.42% жилье


### Шаг 4. Общий вывод

Мотивация и причины невозврата у исследованных групп разная. Отсутствует прямая зависимость возвращаемости кредита от уровня дохода, что видно по заемщикам с высокими финансовыми возможностями. Увеличение расходов на содержание иждевенцев(детей) имеет определенное влияние, связанное с приоритетом обеспечения первоочередных нужд ребенка. Выделяется повышенная ответственность по возврату кредита у тех, кто принял обдуманное решение родить третьего ребенка, что может объяснить выделение отдельной группы родителей-заемщиков. Люди не готовые взять ответственность по Семейному Кодексу РФ, с большей долей вероятности не готовы брать ответственность и по другим обязательствам. Выбор цели кредита также определяет риск его невозвращаемости, т к кредит на автомобиль и образование зачастую берут люди, имеющие малый опыт финансовых накоплений. 
Таким образом, исходя из выборки можно сделать вывод_ что на возвращаемость кредита влияет социальный портрет заемщика, который по определенным критериям может быть отнесен к группам которые способны принять ответственное решение, не увеличивая финансовую нагрузку, способные правильно определить цель кредита.

